In [91]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [92]:
import pandas as pd 
df = pd.read_csv('two node example.csv')
t_set = df['t'].unique()

In [93]:
print(df) 

   t  D_1t  D_2t
0  1     1     1
1  2     5     5
2  3    10    10


In [94]:
# constraints 

#  \sum_{k \in K_1} g_{1,k,t} - f_t \geq d_{1,t}~~\forall t \in T ~~~{\color{blue} \lambda_{1,t}}
def demand1(model, t):
    return (model.g_1_1_t[t] + model.g_1_2_t[t] - model.f_t[t] >= model.d_1t[t])

#   \sum_{k \in K_2} g_{2,k,t} + f_t \geq d_{2,t}~~\forall t \in T,  ~~~{\color{blue} \lambda_{2,t}}
def demand2(model, t):
    return (model.g_2_1_t[t] + model.g_2_2_t[t]+ model.f_t[t] >= model.d_2t[t])

#  g_{i,k,t} -  h_{i,k} \leq 0 ~~\forall i, \forall t \in T~~~{\color{blue}\pi_{i,k,t}}
# i = 1, k = 1 
def generator_capacity_1_1 (model,t): 
    return (model.g_1_1_t [t] - model.h_1_1 <= 0) 
# i =2 , k = 1 
def generator_capacity_2_1 (model, t): 
    return (model.g_2_1_t[t] - model.h_2_1 <= 0) 
# i =1 , k =2 
def generator_capacity_1_2 (model, t): 
    return (model.g_1_2_t[t] - model.h_1_2 <= 0 )
# i = 2, k = 2 
def generator_capacity_2_2 (model, t): 
    return( model.g_2_2_t[t] - model.h_2_2 <= 0 )

# transmission capacity 
def transmission_capacity_positive (model, t): 
   return (model.f_t [t] - model.F <= 0)

def transmission_capacity_negative (model, t): 
   return( - model.f_t [t] - model.F <= 0)

In [95]:
# create a model 
model = pyomo.ConcreteModel()

# series 
model.nt = pyomo.Param(initialize=len(df))
model.T = pyomo.Set(initialize=t_set)

# Parameters
# c^H_{i,k} 
model.c_H_1_1 = pyomo.Param(initialize = 1.0) # i = 1, k = 1
model.c_H_1_2 = pyomo.Param(initialize = 2.0) # i = 1, k = 2 
model.c_H_2_1 = pyomo.Param(initialize = 3.0) # i = 2, k = 1 
model.c_H_2_2 = pyomo.Param(initialize = 4.0) # i = 2, k = 2 
#c^G_{i,k} 
model.c_G_1_1 = pyomo.Param(initialize = 4.0) # i = 1, k = 1
model.c_G_1_2 = pyomo.Param(initialize = 3.0) # i = 1, k = 2 
model.c_G_2_1 = pyomo.Param(initialize = 2.0) # i = 2, k = 1 
model.c_G_2_2 = pyomo.Param(initialize = 1.0) # i = 2, k = 2 

model.c_F = pyomo.Param (initialize =3) #c^F
model.d_1t = pyomo.Param(model.T, initialize = {t: df.loc[(df['t']==t), 'D_1t' ].values[0] for t in t_set}) 
model.d_2t = pyomo.Param(model.T, initialize = {t: df.loc[(df['t']==t), 'D_2t' ].values[0] for t in t_set}) 

# Variables
# g_{1,k,t} 
model.g_1_1_t = pyomo.Var(model.T, domain = pyomo.NonNegativeReals) #g_{1,1,t}, k = 1 
model.g_1_2_t = pyomo.Var (model.T, domain = pyomo.NonNegativeReals) #g_{1,2,t}, k = 2
# g_{2,k,t} 
model.g_2_1_t = pyomo.Var(model.T, domain = pyomo.NonNegativeReals) #g_{2,1,t}, k = 1 
model.g_2_2_t = pyomo.Var (model.T, domain = pyomo.NonNegativeReals) #g_{2,2,t}, k = 2
#h_{i,k} - node i, tech k 
model.h_1_1 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,1}, i = 1, k = 1
model.h_1_2 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,2}, i = 1, k = 2 
model.h_2_1 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,1}, i = 2, k = 1
model.h_2_2 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,2}, i = 2, k = 2 
model.f_t = pyomo.Var(model.T, domain = pyomo.Reals) # +ve is node 1 to node 2, -ve is node 2 to node 1 
model.F = pyomo.Var ( domain = pyomo.NonNegativeReals) 

# objective function 
#  \min ~ \sum_{i} \left( \sum_{k}  \left(  c^H_{i,k} h_{i,k} + \sum_{t} c^G_{i,k}g_{i,k,t} \right)\right) + c^F F
def objective_func(model): 
    return (
        model.c_H_1_1 * model.h_1_1
        + model.c_H_1_2 * model.h_1_2
        + model.c_H_2_1 * model.h_2_1
        + model.c_H_2_2 * model.h_2_2
        + sum(model.c_G_1_1 * model.g_1_1_t[t] for t in model.T)
        + sum(model.c_G_1_2 * model.g_1_2_t[t] for t in model.T)
        + sum(model.c_G_2_1 * model.g_2_1_t[t] for t in model.T)
        + sum(model.c_G_2_2 * model.g_2_2_t[t] for t in model.T)
        + model.c_F * model.F
    )


model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)

# Constraints 
model.demand1Constraint = pyomo.Constraint (model.T, rule = demand1) 
model.demand2Constraint = pyomo.Constraint (model.T, rule = demand2) 
model.generatorCapacity11 = pyomo.Constraint (model.T, rule = generator_capacity_1_1) 
model.generatorCapacity21 = pyomo.Constraint (model.T, rule = generator_capacity_2_1) 
model.generatorCapacity12 = pyomo.Constraint (model.T, rule = generator_capacity_1_2) 
model.generatorCapacity22 = pyomo.Constraint (model.T, rule = generator_capacity_2_2) 
model.transmissionCapacityPositive = pyomo.Constraint (model.T,  rule = transmission_capacity_positive) 
model.transmissionCapacityNegative = pyomo.Constraint(model.T, rule= transmission_capacity_negative) 

#objective - solve 
result = solver.solve(model)
print(model.objective())
print(result.solver.status)
print(result.solver.termination_condition)    
print(model.display())

123.0
ok
optimal
Model unknown

  Variables:
    g_1_1_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    g_1_2_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   4.0 :  None : False : False : NonNegativeReals
          3 :     0 :  11.0 :  None : False : False : NonNegativeReals
    g_2_1_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    g_2_2_t : Size=3, Index=T
        Key :